<a href="https://colab.research.google.com/github/alexisrisaldi/prueba/blob/master/Separaci%C3%B3n%20de%20ciudades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load cleaned Excel file
df = pd.read_excel("output_excel.xlsx", parse_dates=["Snapshot date"])

# Cargar el archivo con los identificadores a unir
df_unificar = pd.read_excel("Unificar.xlsx")


In [ ]:
#verificar si ambos tienen el mismo tipo de dato
df['Survey Id'].dtype
df_unificar['Snapshot Id'].dtype

dtype('int64')

In [ ]:
# Renombrar columna para que coincida con 'Survey Id'
df_unificar = df_unificar.rename(columns={"Snapshot Id": "Survey Id"})

In [ ]:
# Display the first few rows of the DataFrame 'df_unificar'
print(df_unificar.head())


   Survey Id  Unique Identifier
0     467823              97081
1     467824              97121
2     467825              97066
3     467826              97109
4     467827              97092


In [ ]:
# Realizar el left join y sobrescribir el DataFrame original
df = pd.merge(df, df_unificar, on="Survey Id", how="left")

In [ ]:
#verificar si se se hizo el join
print(df.head())

In [ ]:
# Get actual survey sequences for each family
def check_missing_surveys(group):
    expected = list(range(1, group['Survey Number'].max() + 1))
    actual = sorted(group['Survey Number'].unique())
    missing = set(expected) - set(actual)
    if missing:
        return pd.Series({'Missing Surveys': sorted(missing)})
    return pd.Series({'Missing Surveys': None})

missing_surveys = df.groupby("Family Id").apply(check_missing_surveys).dropna()
missing_surveys.reset_index(inplace=True)

missing_surveys.to_excel("families_with_missing_surveys.xlsx", index=False)

In [ ]:
survey_cols = [
    "accessToCredit", "accessToEntertainment", "accessToHealthServices", "accessToInternet",
    "alimentation", "autonomyDecisions", "awarenessOfNeeds", "capacityToPlanAndBudget",
    "clothingAndFootwear", "comfortOfTheHome", "covid_homeEducation", "covid_technology",
    "diversifiedSourcesOfIncome", "documentation", "drinkingWaterAccess", "electricityAccess",
    "familySavings", "garbageDisposal", "income", "influenceInPublicSector", "insurance",
    "middleEducation", "properKitchen", "readAndWrite", "refrigerator",
    "registeredToVoteAndVotesInElections", "safeBathroom", "safeHouse", "safety",
    "securityOfProperty", "selfEsteem", "separateBedrooms", "socialCapital",
    "unpollutedEnvironment", "vaccinations"
]

# Filter rows where any survey value is missing
rows_with_missing_indicators = df[df[survey_cols].isnull().any(axis=1)]

rows_with_missing_indicators.to_excel("rows_with_missing_indicators.xlsx", index=False)

In [ ]:
df = df.dropna(subset=survey_cols).reset_index(drop=True)

In [ ]:
# Sort and calculate date gaps
df = df.sort_values(["Family Id", "Snapshot date"])
df["Days_Since_Last_Survey"] = df.groupby("Family Id")["Snapshot date"].diff().dt.days

# Keep only surveys spaced at least 90 days apart
filtered_df = df[(df["Days_Since_Last_Survey"].isna()) | (df["Days_Since_Last_Survey"] >= 90)]


In [ ]:
filtered_df.to_excel("final_cleaned_survey_data.xlsx", index=False)


Segregating families by the number of survey's they took.

In [ ]:
# Count how many surveys each family took (after all cleaning)
family_counts = df.groupby("Family Id").size().reset_index(name="Survey_Count")

# Merge counts back to the main dataframe
df_with_counts = df.merge(family_counts, on="Family Id")

# Loop through each survey count and export to individual files
for count in sorted(df_with_counts["Survey_Count"].unique()):
    group_df = df_with_counts[df_with_counts["Survey_Count"] == count]
    filename = f"families_with_{count}_surveys.xlsx"
    group_df.to_excel(filename, index=False)

print("Exported families into separate files by number of surveys taken.")


Exported families into separate files by number of surveys taken.


In [ ]:
print(sorted(df_with_counts["Survey_Count"].unique()))


[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(15), np.int64(17), np.int64(23)]
